In [1]:
import pandas as pd
import numpy as np
import folium
from geojson import cities

In [2]:
df = pd.read_csv('contratos_com_coordenadas.csv', sep=';')
df.head()

,data_abertura,email,telefone,nome_empresarial,nome_fantasia,logradouro,numero,complemento,cep,bairro,...,vencimento,cnpj,razao_social,cod_cnae,descricao_cnae,cod_natureza,descricao_natureza,endereco_completo,latitude,longitude
0,2002-06-28,ADRIANO@AEROTUR.COM.BR,84 32202950,AEROTUR SERVICOS DE VIAGENS LTDA,NaN,R APODI,583,SALA 02 - 1 ANDAR,,TIROL,...,2018-11-20,5120923000109,AEROTUR SERVIÇOS DE VIAGENS LTDA,79112,Agências de viagens,2062,Sociedade Empresária Limitada,"R. Apodi, 583 - SALA 02 - 1 ANDAR - Tirol, Nat...",-5.791534,-35.200320
1,1999-11-30,PRESCOFIL@BOL.COM.BR,84 32724098 84 6130111,MAC DE MELO,MEGADIESEL,AV PILOTO PEREIRA TIM,2260,NaN,,CENTRO,...,2019-10-03,3538129000146,M.A.C DE MELO,0,Sem informação,2135,Empresário (Individual),"Av. Piloto Pereira Tim, 2260 - Centro, Parnami...",-5.910796,-35.261644
2,1984-11-05,CARBOX@CARBOX.COM.BR,84 32170106 84 32170106,CARBOX - MONTEIRO & MESQUITA LTDA,CARBOX,AV DAO SILVEIRA,4412,NaN,,CANDELARIA,...,2019-10-03,9080300000100,MONTEIRO & MESQUITA LTDA EPP,0,Sem informação,2062,Sociedade Empresária Limitada,"Av. Dão Silveira, 4412 - Candelária, Natal - R...",-5.856718,-35.212661
3,2008-08-27,CARBOX@CARBOX.COM.BR,84 32076913 84 32139025,CARBOX LTDA,NaN,AV DAO SILVEIRA,4200,NaN,,CANDELARIA,...,2019-10-03,10334942000160,CARBOX LTDA - ME,0,Sem informação,2062,Sociedade Empresária Limitada,"Av. Dão Silveira, 4200 - Candelária, Natal - R...",-5.856470,-35.212670
4,2008-08-15,TOTALMIDIAEVENTOS@HOTMAIL.COM,84 32119441 84 32119441,TOTAL MULTIMIDIA LOCACAO E EVENTOS LTDA,TOTAL MULTIMIDIA,R NASCIMENTO FERNANDES,2145,NaN,,LAGOA NOVA,...,2019-01-04,10298485000103,TOTAL MULTIMIDIA LOCAÇÃO E EVENTOS LTDA-ME,77390,Aluguel de máquinas e equipamentos não especif...,2062,Sociedade Empresária Limitada,"R. Nascimento Fernandes, 2145 - Lagoa Nova, Na...",-5.816316,-35.200792


In [3]:
natureza_icons = {
    2135: 'female', #Empresário (Individual)
    2062: 'building', #Sociedade Empresária Limitada
    2305: 'male', #Empresa Individual de Responsabilidade Limitada
    2054: 'industry', #Sociedade Anônima Fechada
}

Outra hipótese que eu tinha era que existiam empresas acumuladas em um mesmo prédio ou muito próximas uma da outra, o que poderia indicar atividades suspeitas.

Apenas duas empresas apresentaram comportamento próximo do que eu esperava: DirectCOM Informática e Comércio e Serviços em Elevadores.

In [4]:
m = folium.Map(
    location=[-15.765379, -47.968776],
    zoom_start=4
)
for place in df[['nome_empresarial', 'latitude', 'longitude', 'cod_natureza', 'descricao_natureza']].values:
    folium.Marker(
        [place[1], place[2]],
        popup='<b>{}</b><br>{}'.format(place[0], place[4]),
        icon=folium.Icon(icon=natureza_icons.get(place[3], 'university'), prefix='fa')
    ).add_to(m)
m

In [5]:
df_total = df.groupby('municipio')['valor'].sum().sort_values(ascending=False).to_frame().reset_index()
df_total.head()

,municipio,valor
0,NATAL,89604477.91
1,FORTALEZA,28566299.86
2,RECIFE,6357900.00
3,SÃO PAULO,6241389.55
4,UBERLÂNDIA,2499500.00


In [6]:
geojson = {
    'type': 'FeatureCollection',
    'features': cities
}

Complementar a hipótese anterior, uma grande quantidade de dinheiro concentrada em um único lugar também indicaria atividade suspeita. Entretanto, com os dados fornecidos pouco se poderia inferir.

In [7]:
m = folium.Map(
    location=[-5.826592, -35.212558],
    zoom_start=7,
    tiles='OpenStreetMap'
)

# create a threshold of legend
threshold_scale = np.linspace(df_total['valor'].min(),
                              df_total['valor'].max(), 5, dtype=int).tolist()


m.choropleth(
    geo_data=geojson,
    data=df_total,
    columns=['municipio', 'valor'],
    key_on='feature.properties.name',
    fill_color='PuRd',
    fill_opacity=0.7,
    legend_name='Valor em reais Estado',
    highlight=True,
    threshold_scale = threshold_scale
)

m

Análise FINAL/GERAL:

